In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [3]:
X_train, Y_train = read_csv('train.csv')
X_test, Y_test = read_csv('test.csv')

In [4]:
maxLen = len(max(X_train, key=len).split())

In [5]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [6]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype = 'int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    X = LSTM(units = 128, return_sequences = True)(embeddings)
    X = Dropout(rate = 0.5)(X)
    X = LSTM(units = 128, return_sequences = False)(X)
    X = Dropout(rate = 0.5)(X)
    X = Dense(units = 5)(X)
    X = Activation('softmax')(X)
    model = Model(inputs = sentence_indices, outputs = X)
    return model

In [8]:
model = Emojify((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645 

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
def sentences_to_indices(X, word_to_index, max_len):    
    m = X.shape[0]
    X_indices = np.zeros((X.shape[0], max_len))
    for i in range(m):
        sentence_words = X[i].lower().strip().split()
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j + 1
    return X_indices

In [11]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [12]:
#training the model
model.fit(X_train_indices, Y_train_oh, epochs = 40, batch_size = 33, shuffle=True)


Epoch 1/40
132/132 [==============================] - 2s 16ms/step - loss: 1.5950 - accuracy: 0.2424
Epoch 2/40
132/132 [==============================] - 0s 1ms/step - loss: 1.5215 - accuracy: 0.3258
Epoch 3/40
132/132 [==============================] - 0s 1ms/step - loss: 1.4582 - accuracy: 0.3485
Epoch 4/40
132/132 [==============================] - 0s 1ms/step - loss: 1.3713 - accuracy: 0.4773
Epoch 5/40
132/132 [==============================] - 0s 1ms/step - loss: 1.2861 - accuracy: 0.5379
Epoch 6/40
132/132 [==============================] - 0s 1ms/step - loss: 1.1215 - accuracy: 0.5833
Epoch 7/40
132/132 [==============================] - 0s 1ms/step - loss: 1.0352 - accuracy: 0.6364
Epoch 8/40
132/132 [==============================] - 0s 1ms/step - loss: 0.8442 - accuracy: 0.7424
Epoch 9/40
132/132 [==============================] - 0s 1ms/step - loss: 0.7382 - accuracy: 0.7197
Epoch 10/40
132/132 [==============================] - 0s 2ms/step - loss: 0.6773 - accuracy: 0.77

In [13]:
#testing the model on test data
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 3ms/step
Test accuracy =  0.7857142686843872


In [14]:
#let's see the mislabelled texts
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: he got a very nice raise	❤️
Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😄 prediction: he is a good friend	❤️
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😄 prediction: you brighten my day	❤️
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😞 prediction: My life is so boring	❤️
Expected emoji:😄 prediction: will you be my valentine	❤️
Expected emoji:😄 prediction: I like to laugh	❤️
Expected emoji:😄 prediction: What you did was awesome	😞
Expected emoji:😞 prediction: go away	⚾
Expected emoji:😄 prediction: You deserve this nice prize	❤️


In [15]:
#test your own text
x_test = np.array(["i love you"])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

i love you ❤️
